In [1]:
import torch
import ctypes

# Load shared library
lib = ctypes.CDLL('./cuda_tools/libipc_tensor_tool.so')
lib.open_ipc_tensor.argtypes = [ctypes.c_void_p, ctypes.c_int]
lib.open_ipc_tensor.restype = ctypes.c_void_p

lib.close_ipc_tensor.argtypes = [ctypes.c_void_p]
lib.close_ipc_tensor.restype = ctypes.c_int


DTYPE_SIZE = {
    torch.float32: 4,
    torch.int32: 4,
    torch.int64: 8,
    torch.float64: 8,
    torch.uint8: 1,
    # add more if needed
}

DTYPE_MAP = {
    'torch.float32':torch.float32,
    'torch.int32':torch.int32,
    'torch.int64':torch.int64,
    'torch.float64':torch.float64,
    'torch.uint8':torch.uint8,
    'torch.float16':torch.float16,
    'torch.bfloat16':torch.bfloat16,
}

def restore_tensor(ipc_handle_bytes: bytes, shape, dtype=torch.float32, device=0):
    if len(ipc_handle_bytes) != 64:
        raise ValueError("Invalid IPC handle size")

    dtype = DTYPE_MAP.get(dtype, None)
    if dtype not in DTYPE_SIZE:
        raise ValueError(f"Unsupported dtype: {dtype}")

    handle_buf = ctypes.create_string_buffer(ipc_handle_bytes, 64)
    dev_ptr = lib.open_ipc_tensor(handle_buf, device)

    if not dev_ptr:
        raise RuntimeError("Failed to open IPC handle")

    numel = torch.prod(torch.tensor(shape)).item()
    nbytes = numel * DTYPE_SIZE[dtype]

    # Wrap the pointer as a ctypes pointer of the right type
    # (important: we cast to ctypes type matching dtype)
    ptr_type = ctypes.POINTER(ctypes.c_float)  # default
    if dtype == torch.float32:
        ptr_type = ctypes.POINTER(ctypes.c_float)
    elif dtype == torch.int32:
        ptr_type = ctypes.POINTER(ctypes.c_int32)
    elif dtype == torch.int64:
        ptr_type = ctypes.POINTER(ctypes.c_int64)
    elif dtype == torch.float64:
        ptr_type = ctypes.POINTER(ctypes.c_double)
    elif dtype == torch.uint8:
        ptr_type = ctypes.POINTER(ctypes.c_uint8)
    else:
        raise ValueError(f"Unsupported dtype for ctypes cast: {dtype}")

    typed_ptr = ctypes.cast(dev_ptr, ptr_type)

    # Use torch.from_blob (no ownership)
    t = torch.frombuffer(
        (ctypes.c_char * nbytes).from_address(dev_ptr),
        dtype=dtype
    ).view(*shape).to(f'cuda:{device}')
    lib.close_ipc_tensor(dev_ptr)
    return t

In [ ]:
from flask import Flask, request, jsonify, Response
import json

app = Flask(__name__)

@app.route('/upload', methods=['POST'])
def upload():
    # 1. 获取二进制数据（如文件）
    byte_data = request.files['byte_data'].read()
    
    # 2. 获取字典数据（JSON 格式）
    map_data = json.loads(request.form['map_data']) 
    
    t = restore_tensor(byte_data, shape=map_data["shape"], dtype=map_data["dtype"], device=int(map_data["device"]))
    print(t)
    # 3. 返回结果
    response = {
        'message':"ok",
        'restored_tensor':t.cpu().tolist(),  # Convert to list for JSON serialization
    }
    
    return jsonify(response)

if __name__ == '__main__':
    app.run(port=1177)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:1177
Press CTRL+C to quit


[2025-05-14 02:41:30,191] ERROR in app: Exception on /upload [POST]
Traceback (most recent call last):
  File "/root/vllm_test/myllm/lib/python3.10/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "/root/vllm_test/myllm/lib/python3.10/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/root/vllm_test/myllm/lib/python3.10/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "/root/vllm_test/myllm/lib/python3.10/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/tmp/ipykernel_14957/2766625509.py", line 14, in upload
    t = restore_tensor(byte_data, shape=map_data["shape"], dtype=getattr(torch, map_data["dtype"]), device=int(map_data["device"]))
  File "/root/vllm_test/myllm/lib/python3.10/site-packages